# 使用方法

## 1. 数据集的加载

In [ ]:
from modules.classes import Dataset

source = './images/' # 装有图像的文件夹
dataset = Dataset(
    source, # 数据集源，可以是一个或多个（用列表包装）(i) 文件夹目录、(ii) 图像文件路径、(iii) json/csv 文件路径
    read_caption=True, # 是否读取 图像名.txt 作为标注，仅对源 (i) 和 (ii) 有效
    formalize_caption=True, # 是否自动格式化字符串
    recur=True, # 是否递归读取子文件夹，仅对源 (i) 有效
    verbose=True, # 是否打印详细信息，建议开启
)

## 2. 数据集的导出

In [ ]:
dataset.to_csv('./database/my-database.csv') # 保存为 csv 数据库文件

In [ ]:
dataset.to_json('./database/my-database.json') # 保存为 json 数据库文件

In [ ]:
dataset.to_txts() # 写入为 图像名.txt 数据库文件

## 3. 数据集的可视化

In [ ]:
print(dataset)

In [ ]:
for img_key, img_info in dataset.items():
    print(img_key, '\n  '.join([f'{k}: {v}' for k, v in img_info.dict().items()]))

## 4. 数据集的操作
数据在数据集中的存储用 ImageInfo 图像信息类来表示，image_info 类中包含了图像的路径、图像的标注信息、图像的宽高等信息。

假如 a 是一个 ImageInfo 类的对象，那么它具有如下属性：
1. **a.image_path: \[可修改\] 图像的文件路径**
2. **a.caption: \[可修改\] 数据标注**
3. a.original_size: 图像的原始尺寸
4. a.key: 数据的唯一标识符，即其文件名称（不包含扩展名）
5. a.stem: 同 a.key
6. a.suffix: 图像文件的扩展名
7. a.category: 图像的类别，即其所在的文件夹名称
8. a.source: 图像的来源，即其所在的文件夹的所在文件夹的**路径**

其中，a.caption 是一个 Caption 类的对象，它又具有如下属性：
1. a.caption.tags: 标注的 tag 列表
2. a.caption.caption: 标注的文本，用逗号加空格分隔标签
3. a.caption.artist: 图像的作者
4. a.caption.quality: 图像的质量
5. a.caption.styles: 图像的风格
6. a.caption.characters: 图像中的角色
修改上述任何一个属性将会自动更新其它属性，例如修改 a.caption.caption 将会自动更新 a.caption.tags, a.caption.artist 等其他几个属性。

数据集批量操作的核心是 apply_map 函数。apply_map 接受的参数为另一个变换函数 func 和 func 的其他参数。
其中，func 必须接受一个 ImageInfo 类的对象作为其第一个参数，且返回一个 ImageInfo 类的对象，作为变换后的数据。
以下三个示例展示了 apply_map 的用法

In [ ]:
# 示例一：为所有标注添加一个前缀 `my tag`

def my_transform(image_info):
    image_info.caption = 'my tag' + image_info.caption
    return image_info
dataset.apply_map(my_transform)

In [ ]:
# 示例二：功能与示例一相同，但是使用了 `tag` 作为额外参数，便于修改

def my_transform(image_info, tag):
    image_info.caption = tag + image_info.caption
    return image_info

dataset.apply_map(my_transform, 'my tag') # 对每个图像信息对象执行 my_transform 函数

In [ ]:
# 示例三：格式化并优化所有标注
from modules import tagging
tagging.init_priority_tags() # 初始化优先标签

def my_transform(image_info):
    caption = image_info.caption
    if caption is None:
        return image_info

    # 以下为格式化和优化标注的示例，每一行均为原子操作，允许你单独修改
    caption = caption.formalized() # 自动格式化，包括 tag 空格化、括号转义、属性标签提取等
    caption = caption.unique() # 去除重复标签
    caption -= 'signature, low quality, watermark' # 去除指定标签
    caption = caption @ tagging.PRIORITY_REGEX # tag 优先级排序
    caption = caption.deovlped() # tag 语义去重
    
    image_info.caption = caption
    return image_info

dataset.apply_map(my_transform)

In [ ]:
# 示例四：去除角色特征
from modules.tools import make_character_feature_table
character_feature_table = make_character_feature_table(dataset) # 生成角色特征表

In [ ]:
# 打印角色特征表
for char_tag, table in character_feature_table.items():
    print(char_tag + ':\n ', "\n  ".join([f"[{k}]{''.join([' ']*(max(0, 20-(len(k)))))}: 出现次数: {v[0]} | 频率: {v[1]}" for k, v in table.items()]))

In [ ]:
# 去除角色特征

def my_transform(image_info):
    caption = image_info.caption
    if caption is None:
        return image_info
    caption = caption.defeatured(character_feature_table, 0.3) # 去除角色特征
    image_info.caption = caption
    return image_info

dataset.apply_map(my_transform)

In [ ]:
from modules.tools import run_tagger

run_tagger(
    dataset,
    save_path='./database/my-database.csv', # 保存路径
    save_every_n_steps=100,
    verbose=True,
)